# **1. Configuración del Ambiente**


---

In [166]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from scipy.stats import randint
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
global df_traffic, resultados, modelo, modelo_clasificacion

# **2. Creación de Modelo de Regresión Lineal**


---

In [167]:
df_traffic = pd.read_csv('Classifier.csv', sep=';')
df_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12283 entries, 0 to 12282
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   visitNumber         12283 non-null  int64  
 1   browser             12283 non-null  int64  
 2   operatingSystem     12283 non-null  int64  
 3   deviceCategory      12283 non-null  int64  
 4   continent           12283 non-null  int64  
 5   country             12283 non-null  int64  
 6   city                12283 non-null  int64  
 7   networkDomain       12283 non-null  int64  
 8   source              12283 non-null  int64  
 9   medium              12283 non-null  int64  
 10  keyword             12283 non-null  int64  
 11  referralPath        12283 non-null  int64  
 12  adContent           12283 non-null  int64  
 13  pageviews           12283 non-null  int64  
 14  newVisits           12283 non-null  float64
 15  transactionRevenue  12283 non-null  float64
 16  year

In [168]:
X = df_traffic.drop(columns=['transactionRevenue'])
y = df_traffic['transactionRevenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


params = {
    'objective': 'regression',       # Problema de regresión
    'metric': 'mse',                 # Métrica de evaluación: Error cuadrático medio
    'num_leaves': 31,                # Número máximo de hojas en un árbol
    'learning_rate': 0.1,            # Tasa de aprendizaje
    'max_depth': -1,                 # Profundidad máxima de cada árbol (-1 significa sin límite)
    'min_child_samples': 20,         # Número mínimo de muestras necesarias para formar una nueva partición en un nodo
    'reg_alpha': 0.0,                # Parámetro de regularización L1 (alpha)
    'reg_lambda': 0.0,               # Parámetro de regularización L2 (lambda)
    'n_estimators': 100,             # Número de árboles en el conjunto
    'bagging_fraction': 0.8,         # Fracción de muestras para construir cada árbol (subsampling)
    'feature_fraction': 0.8,         # Fracción de características para construir cada árbol (subsamplling de características)
    'bagging_freq': 5,                # Frecuencia de subsampling (se realiza cada 5 iteraciones)
    'force_col_wise': True          # Forzar el modo de entrenamiento de columnas
}

lgb_regressor = lgb.LGBMRegressor(**params)
lgb_regressor.fit(X_train, y_train)
y_pred = lgb_regressor.predict(X_test)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Total Bins 914
[LightGBM] [Info] Number of data points in the train set: 9826, number of used features: 20
[LightGBM] [Info] Start training from score 1.488793
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [169]:
#definiendo el K - número de subconjuntos
cv = KFold(n_splits = 5, shuffle=True)

In [172]:
lgbm_regressor = lgb.LGBMRegressor()

# Definir los parámetros
param_lgbm_regressor = {
    'num_leaves': [31],
    'learning_rate': [0.1],
    'max_depth': [-1],
    'min_child_samples': [20],
    'reg_alpha': [0.0],
    'reg_lambda': [0.0],
    'n_estimators': [100],
    'bagging_fraction': [0.8],
    'feature_fraction': [0.8],
    'bagging_freq': [5],
    'force_col_wise': [True]
}

# Configurar RandomizedSearchCV
randomized_search_lgbm = RandomizedSearchCV(
    estimator=lgbm_regressor,
    param_distributions=param_lgbm_regressor,
    n_iter=1,  # Solo se ajustará una vez
    cv=5,  # Número de divisiones de validación cruzada
    scoring='neg_mean_squared_error',  # Métrica de evaluación
    n_jobs=-1,  # Utilizar todos los núcleos de la CPU
    random_state=42  # Semilla aleatoria para reproducibilidad
)

In [173]:
# Ajustar el modelo
randomized_search_lgbm.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Total Bins 914
[LightGBM] [Info] Number of data points in the train set: 9826, number of used features: 20
[LightGBM] [Info] Start training from score 1.488793
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

RandomizedSearchCV(cv=5, estimator=LGBMRegressor(), n_iter=1, n_jobs=-1,
                   param_distributions={'bagging_fraction': [0.8],
                                        'bagging_freq': [5],
                                        'feature_fraction': [0.8],
                                        'force_col_wise': [True],
                                        'learning_rate': [0.1],
                                        'max_depth': [-1],
                                        'min_child_samples': [20],
                                        'n_estimators': [100],
                                        'num_leaves': [31], 'reg_alpha': [0.0],
                                        'reg_lambda': [0.0]},
                   random_state=42, scoring='neg_mean_squared_error')

In [174]:
# Obtener los mejores parámetros y la mejor puntuación
best_params = randomized_search_lgbm.best_params_

In [175]:
#el mejor score
best_score = randomized_search_lgbm.best_score_

In [176]:
print("Mejores parámetros:", best_params)
print("Mejor puntuación:", best_score)

Mejores parámetros: {'reg_lambda': 0.0, 'reg_alpha': 0.0, 'num_leaves': 31, 'n_estimators': 100, 'min_child_samples': 20, 'max_depth': -1, 'learning_rate': 0.1, 'force_col_wise': True, 'feature_fraction': 0.8, 'bagging_freq': 5, 'bagging_fraction': 0.8}
Mejor puntuación: -344.9604636926821


In [177]:
# Crear una nueva instancia de LGBMRegressor con los mejores parámetros encontrados
best_lgbm = lgb.LGBMRegressor(**randomized_search_lgbm.best_params_)

# Entrenar el modelo con los datos de entrenamiento
best_lgbm.fit(X_train, y_train)

y_pred = best_lgbm.predict(X_test)

# Calcular el MSE
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calcular R^2
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

# Calcular RMSE
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Total Bins 914
[LightGBM] [Info] Number of data points in the train set: 9826, number of used features: 20
[LightGBM] [Info] Start training from score 1.488793
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
Mean Squared Error: 223.35476659731054
R^2 Score: 0.47984099519370427
Root Mean Squared Error: 14.94505826677536


In [178]:
model_scores = {
    'model': 'LightGBMRegressor',
    'mse': mse,
    'r2': r2,
    'rmse': rmse
}
print(model_scores)

{'model': 'LightGBMRegressor', 'mse': 223.35476659731054, 'r2': 0.47984099519370427, 'rmse': 14.94505826677536}


In [179]:
def save_scores(model_scores: dict):
  status = []
  #Utilizamos un try-except en caso de que el archivo no exista que cree uno
  try:
    #Si el archivo existe continúa por acá
    scores = pd.read_csv('scores.csv', sep=';')
    status.append('El archivo existe')
    exist = model_scores['model'] in scores['model'].values
    #Validamos si el modelo ya existe en el archivo de scores
    if (exist):
      #Si el modelo ya existe reemplazamos sus valores
      status.append('El modelo existe')
      criteria = scores['model'] == model_scores['model']
      index = scores[criteria].index[0]
      scores.iloc[index] = model_scores
      status.append('Se reemplazaron los valores del modelo')
    else:
      #Si el modelo no existe lo agregamos
      status.append('El modelo no existe')
      df_model_scores = pd.DataFrame(model_scores, index=[0])
      scores = pd.concat([scores, df_model_scores], ignore_index=True)
      status.append('Se añadió modelo nuevo y sus valores')
  except:
    #Si no existe el archivo lo creamos y cargamos los scores del modelo
    status.append('El archivo no existe')
    scores = pd.DataFrame(model_scores, index=[0])

  status.append('Se sobrescrive el archivo scores.csv con valores nuevos')
  scores.to_csv('scores.csv', sep=';', index=False)
  return status

In [180]:
save_scores(model_scores)

['El archivo existe',
 'El modelo existe',
 'Se reemplazaron los valores del modelo',
 'Se sobrescrive el archivo scores.csv con valores nuevos']